In [ ]:
from planet4 import clustering, io, markings, stats, plotting
from pathlib import Path

In [ ]:
db = io.DBManager()

In [ ]:
obsid = 'ESP_012079_0945'

In [ ]:
data = db.get_image_name_markings(obsid)

In [ ]:
image_ids = data.image_id.unique()

In [ ]:
subdata = data[data.image_id.isin(image_ids[:5])]

In [ ]:
subdata.shape

In [ ]:
from ipyparallel import Client
c = Client()

In [ ]:
# At the beginning of the notebook
import logging
logger = logging.getLogger()
assert len(logger.handlers) == 1
logger.addHandler(logging.StreamHandler())
handler = logger.handlers[1]
handler.setLevel(logging.DEBUG)

In [ ]:
def process_imgid(id_, dynamic=True, angle=True, distance=False, radius=False,
                  data=None, eps=15, normalize=False):
    import matplotlib.pyplot as plt
    from planet4 import plotting, clustering
    from pathlib import Path
    path = io.p4data() / ('pipelinecheck7_eps'+str(eps))
    path.mkdir(exist_ok=True)
    cm = clustering.ClusteringManager(output_dir=path,
                                      do_dynamic_min_samples=dynamic,
                                      include_angle=angle,
                                      include_distance=distance,
                                      include_radius=radius,
                                      eps=eps, normalize=normalize)
    cm.cluster_image_id(id_, data=data)
    plotting.plot_image_id_pipeline(id_, datapath=path, save=True)
#     plt.close('all')
    return cm

In [ ]:
def process_img_name(image_name, dynamic=True, angle=True, distance=False, radius=False,
                     data=None, eps=15):
    import matplotlib.pyplot as plt
    from planet4 import plotting, clustering, io
    from pathlib import Path
    path = io.p4data() / ('pipelinecheck7_eps' + str(eps))
    path.mkdir(exist_ok=True)
    cm = clustering.ClusteringManager(output_dir=path,
                                      do_dynamic_min_samples=dynamic,
                                      include_angle=angle,
                                      include_distance=distance,
                                      include_radius=radius,
                                      eps=eps)
    cm.cluster_image_name(image_name, data=data)
#     plotting.plot_image_id_pipeline(id_, datapath=path, save=True)
#     plt.close('all')

In [ ]:
%matplotlib inline

In [ ]:
image_ids = ['APF0000b89','APF0000bff', 'APF0000ctr', 'APF0000bud','APF0000fwe']


In [ ]:
process_imgid(image_ids[0], eps=15, angle=False)

In [ ]:
process_img_name(obsid, data=subdata)

In [ ]:
def angle_to_xy(angle):
    x = np.cos(np.deg2rad(angle))
    y = np.sin(np.deg2rad(angle))
    return x,y

In [ ]:
angle_to_xy(1)

In [ ]:
angle_to_xy(359)

In [ ]:
from planet4.dbscan import DBScanner

In [ ]:
angles = pd.Series([30, 1, 359])

In [ ]:
df = pd.DataFrame({'xang':np.cos(np.deg2rad(angles)),
              'yang':np.sin(np.deg2rad(angles))})

df.values*300

In [ ]:
dbscanner = DBScanner(df.values*300, eps=15, min_samples=2)

In [ ]:
dbscanner.reduced_data

In [ ]:
from sklearn.preprocessing import normalize, Normalizer

In [ ]:
Normalizer?

In [ ]:
normalize(df.values*300)

In [ ]:
df

In [ ]:
from astropy import units as u

In [ ]:
wn = 810 / u.cm

In [ ]:
wn

In [ ]:
wn.to(u.nm, equivalencies=u.equivalencies.spectral())

In [ ]:
wl = 245.7 * u.nm

In [ ]:
wl.to(1/u.cm, equivalencies=u.equivalencies.spectral())

In [ ]:
1/245.7e-9

In [ ]:
for image_id in image_ids[:5]:
    print(image_id)
    process_imgid(image_id, data=data)

In [ ]:
plotting.plot_clustered_fans('1dr')

In [ ]:
plotting.plot_clustered_blotches('1dr')

In [ ]:
plotting.plot_finals('1dr')

In [ ]:
from planet4.plotting import plot_clustered_blotches, plot_clustered_fans, plot_finals

In [ ]:
image_id = '1dr'
imgid = markings.ImageID(image_id)
fig, axes = plt.subplots(ncols=2, figsize=(10, 4))
axes = axes.ravel()
for ax in axes:
    imgid.show_subframe(ax=ax)
plot_clustered_fans(image_id, ax=axes[0])
plot_clustered_blotches(image_id, ax=axes[1])


In [ ]:
process_imgid('1dr', dynamic=True, angle=True, distance=True)

In [ ]:
p1 = (221.79, 508.936)
p2 = (232, 517)

In [ ]:
from scipy.linalg import norm

In [ ]:
dp = np.array(p1) - np.array(p2)

In [ ]:
dp

In [ ]:
norm(dp)

In [ ]:
recheck_ids = ['1dn','1k3','1e4','1fe','1aa','225','1pr','19g']
for imid in recheck_ids:
    print(imid)
    cm.cluster_image_id(imid)
    plotting.plot_image_id_pipeline(imid, datapath=path, save=True)

In [ ]:
db = io.DBManager()
data = db.get_image_id_markings('1fe')

In [ ]:
data.classification_id.nunique()

In [ ]:
plotting.plot_finals(imid, _dir=path)

In [ ]:
plotting.plot_raw_blotches(imid)

In [ ]:
from planet4.plotting import blotches_all, fans_all

In [ ]:
import seaborn as sns
sns.set_context('notebook')
blotches_all(imid)

In [ ]:
fans_all(imid)

In [ ]:
lbview = c.load_balanced_view()

In [ ]:
import nbtools.multiprocessing as mptools

In [ ]:
results = lbview.map_async(process_imgid, ids)

In [ ]:
mptools.nb_progress_display(results, ids)